In [4]:
from ultralytics import YOLO
import cv2
import easyocr
import mysql.connector
from datetime import datetime

# Initialize database connection
db = mysql.connector.connect(
    host="localhost",
    user="your_username",
    password="your_password",
    database="your_database"
)
cursor = db.cursor()

# Create table if it doesn't exist
cursor.execute("""
    CREATE TABLE IF NOT EXISTS CarPlates (
        id INT AUTO_INCREMENT PRIMARY KEY,
        plate_number VARCHAR(50),
        timestamp DATETIME
    )
""")

# Load YOLO model
model = YOLO('license_plate_detector.pt')

# Open video file
cap = cv2.VideoCapture('car 1-1.mp4')
if not cap.isOpened():
    print("Error opening video stream or file")

# Initialize EasyOCR reader
reader = easyocr.Reader(['en'])

# Process video frames
while cap.isOpened():
    ret, frame = cap.read()
    if ret:
        results = model(frame)
        
        # Process YOLO results
        for result in results[0].boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = result
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            
            # Crop detected region for OCR
            cr_img = frame[int(y1):int(y2), int(x1):int(x2)]
            ocr_result = reader.readtext(cr_img)
            plate_text = ''.join([res[1] for res in ocr_result if res[2] > 0.5 and len(res[1]) > 2])

            # Display detected text
            cv2.putText(frame, plate_text, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

            # Insert detected plate and timestamp into the database
            if plate_text:
                timestamp = datetime.now()
                cursor.execute("INSERT INTO CarPlates (plate_number, timestamp) VALUES (%s, %s)", (plate_text, timestamp))
                db.commit()
                print(f"Logged to DB: Plate - {plate_text}, Timestamp - {timestamp}")

        # Display the frame
        cv2.imshow('Image', frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

# Release resources
cap.release()
cv2.destroyAllWindows()
cursor.close()
db.close()


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.



0: 384x640 (no detections), 116.8ms
Speed: 9.7ms preprocess, 116.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 171.8ms
Speed: 6.4ms preprocess, 171.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 110.3ms
Speed: 10.1ms preprocess, 110.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 126.4ms
Speed: 10.3ms preprocess, 126.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 133.9ms
Speed: 4.1ms preprocess, 133.9ms inference, 8.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 146.8ms
Speed: 2.9ms preprocess, 146.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 157.9ms
Speed: 4.2ms preprocess, 157.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 158.5ms
Speed: 6.8ms pre